<div style='background: cyan; font-size: 74px; font-weight: bold'>CNN BASICS</div>

In [1]:
import torch
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

<div style='background: cyan; font-size: 74px; font-weight: bold'>DATA INGESTION</div>

In [2]:
transforms = transforms.Compose([
    transforms.Resize((24, 24)),
    transforms.Grayscale(), # Convert an RGB to a grey channel (blank and white)
    transforms.ToTensor()
])

In [3]:
img_dataset = datasets.ImageFolder(root='../datasets/processed/', transform=transforms)
loader = DataLoader(dataset=img_dataset, batch_size=32, shuffle=True)

<div style='background: cyan; font-size: 74px; font-weight: bold'>MODEL DEFINITION</div>

In [4]:
class Mlp(nn.Module):
    def __init__(self, input_size=24*24, hidden_size=128, output_size=38):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(in_features=hidden_size, out_features=output_size) # Output -> [batch_size, num_classes]
        # self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten the inputs
        x = self.relu_1(self.layer_1(x))
        # x = self.softmax(self.layer_2(x)) ⚠️don't apply softmax manually
        return x

<div style='background: cyan; font-size: 74px; font-weight: bold'>TRAINING LOOP</div>

In [5]:
simple_mlp = Mlp()
criterion = nn.CrossEntropyLoss() # Also handles softmax internally
optimizer = torch.optim.Adam(simple_mlp.parameters(), lr=0.001) # Update model params

In [6]:
# Training Loop
num_epochs = 20

for epoch in range(num_epochs):
    simple_mlp.train() # Mostly for models with batch-norm
    running_loss = 0.0

    progress_bar = tqdm(loader, ncols=1000, desc=f"Epoch {epoch+1}/{num_epochs}") # For showing progress bars
    for imgs, labels in progress_bar:
        outputs = simple_mlp(imgs) # Forward pass, data ingestion
        loss = criterion(outputs, labels) # Loss calculation

        optimizer.zero_grad() # Set gradients to zero, cause grads from the previous epoch are retained by default
        loss.backward() # perform backpropagation
        optimizer.step() # Update weights

        progress_bar.set_postfix(loss=loss.item())

        running_loss += loss.item()
    epoch_loss = running_loss / len(loader)
    print(f"   Epoch {epoch+1}, - Avg Loss: {epoch_loss:.4f}") # Track loss for each epoch

Epoch 1/20:   0%|                                                                                             …

   Epoch 1, - Avg Loss: 4.3414


Epoch 2/20:   0%|                                                                                             …

   Epoch 2, - Avg Loss: 4.2330


Epoch 3/20:   0%|                                                                                             …

...

   Epoch 18, - Avg Loss: 4.0351


Epoch 19/20:   0%|                                                                                            …

   Epoch 19, - Avg Loss: 4.0311


Epoch 20/20:   0%|                                                                                            …

   Epoch 20, - Avg Loss: 4.0293


In [ ]:
# logit = torch.tensor([[0.9, 1.2, 3.5, 2.6],
#                       [0.01, 0.78, 2.3, 8.1],
#                       [1.9, 2.81, 5.5, 3.01]]
#                       )
# #apply softmax
# softmax = nn.Softmax(dim=1)
# logit_proba = softmax(logit)
# pred_args = torch.argmax(logit, dim=1)
# ground_truth_labels = torch.tensor([[2, 1, 2]])
# pred = (ground_truth_labels==pred_args)
# pred